In [12]:
!pip install tmdbsimple

In [13]:
import pandas as pd
import numpy as np
import os
import json

In [3]:
akas_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.akas.tsv.gz'
akas_df = pd.read_csv(akas_file, sep='\t', compression='gzip')

/var/folders/h7/97hfnxr93g570s_js7p5lr9h0000gn/T/ipykernel_62406/2025018888.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas_df = pd.read_csv(akas_file, sep='\t', compression='gzip')


In [4]:
akas_df = akas_df[akas_df['region'] == 'US']
akas_df.replace('\\N', np.nan, inplace=True)

In [5]:
basics_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.basics.tsv.gz'
basics_df = pd.read_csv(basics_file, sep='\t', compression='gzip')

/var/folders/h7/97hfnxr93g570s_js7p5lr9h0000gn/T/ipykernel_62406/4154712586.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(basics_file, sep='\t', compression='gzip')


In [6]:
basics_df = basics_df[basics_df['tconst'].isin(akas_df['titleId'])]
basics_df.replace('\\N', np.nan, inplace=True)
basics_df = basics_df.dropna(subset=['runtimeMinutes'])
basics_df = basics_df.dropna(subset=['genres'])
basics_df = basics_df[basics_df['titleType'] == 'movie']
basics_df['startYear'] = basics_df['startYear'].astype(float)
basics_df = basics_df[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <= 2021)]
basics_df = basics_df[~basics_df['genres'].str.contains('Documentary', case=False, na=False)]

In [7]:
ratings_file = '/Users/JoshuaRohn/Desktop/SQL DATA/title.ratings.tsv.gz'
ratings_df = pd.read_csv(ratings_file, sep='\t', compression='gzip')

In [8]:
ratings_df = ratings_df[ratings_df['tconst'].isin(basics_df['tconst'])]
ratings_df.replace('\\N', np.nan, inplace=True)

In [9]:
print("Filtered AKAs Info:")
print(akas_df.info())

print("Filtered Basics Info:")
print(basics_df.info())

print("Filtered Ratings Info:")
print(ratings_df.info())

Filtered AKAs Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1466999 entries, 5 to 37255016
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1466999 non-null  object
 1   ordering         1466999 non-null  int64 
 2   title            1466999 non-null  object
 3   region           1466999 non-null  object
 4   language         4139 non-null     object
 5   types            983972 non-null   object
 6   attributes       47506 non-null    object
 7   isOriginalTitle  1465657 non-null  object
dtypes: int64(1), object(7)
memory usage: 100.7+ MB
None
Filtered Basics Info:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 82109 entries, 34800 to 10215213
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82109 non-null  object 
 1   titleType       82109 non-null  object 
 2   primaryTitle    82

In [10]:
output_folder = 'Data'
os.makedirs(output_folder, exist_ok=True)

In [11]:
akas_df.to_csv(os.path.join(output_folder, 'filtered_akas.csv.gz'), index=False, compression='gzip')
basics_df.to_csv(os.path.join(output_folder, 'filtered_basics.csv.gz'), index=False, compression='gzip')
ratings_df.to_csv(os.path.join(output_folder, 'filtered_ratings.csv.gz'), index=False, compression='gzip')

In [15]:
with open('/Users/JoshuaRohn/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [16]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [17]:
movie = tmdb.Movies(603)

In [18]:
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/oMsxZEvz9a708d49b6UdZK1KAo5.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 69.459,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [19]:
info['budget']

63000000

In [20]:
info['revenue']

463517383

In [21]:
info['imdb_id']

'tt0133093'

In [27]:
releases = movie.releases()
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


In [28]:
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']

In [29]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            info['certification'] = c['certification']
    return info

In [33]:
Avengers = get_movie_with_rating("tt0848228") 
Avengers

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 85.882,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [34]:
Notebook = get_movie_with_rating("tt0332280")
Notebook

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 60.633,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/mevhneWSqbjU22D1MXNd4H9x0r0.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [23]:
filtered_df = pd.read_csv('Data/filtered_basics.csv.gz', compression='gzip')

In [24]:
output_folder = 'FinancialData'
os.makedirs(output_folder, exist_ok=True)

In [25]:
years_to_extract = [2000, 2001]

In [26]:
for year in years_to_extract:
    year_movies = filtered_df[filtered_df['startYear'] == year]
    year_filename = os.path.join(output_folder, f'movies_{year}.csv.gz')
    year_movies.to_csv(year_filename, index=False, compression='gzip')
    print(f"Saved movies for the year {year}.")

Saved movies for the year 2000.
Saved movies for the year 2001.


In [36]:
import time
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 'filtered_basics.csv.gz',
 'filtered_akas.csv.gz',
 '.ipynb_checkpoints',
 'filtered_ratings.csv.gz']

In [37]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [38]:
basics = pd.read_csv('Data/filtered_basics.csv.gz', compression='gzip')

In [39]:
YEARS_TO_GET = [2000,2001]

In [40]:
errors = [ ]

In [42]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)


NameError: name 'tqdm_notebook' is not defined